In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")

In [3]:
bought=pd.read_csv("auction.csv")
dictOfMyPlayers={}
dictOfPlayersBelongingToOtherBidders={}
for idx,player in bought.iterrows():
    if player["Owner"]=="me":
        dictOfMyPlayers[player["Name"]]=player["Price"]
    else:
        dictOfPlayersBelongingToOtherBidders[player["Name"]]=player["Price"]
print(bought.groupby("Owner").sum())

            Price
Owner            
bryan          45
fedemiglia    178
gatti          32
marco          72
me            212
na              4


In [4]:
ampl.read("online-auction.mod")
ampl.readData("fantasy-auction-mantra-online.dat")

	online-auction.mod
	line 39 offset 1150
	Caution: 0-dimensional slice


In [5]:
def suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders):
    plas=ampl.getSet("Players").getValues()
    valuesForOtherPlayersAmpl={}
    valuesForMyPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForMyPlayersAmpl[name]=0
        valuesForOtherPlayersAmpl[name]=0
    budget=500
    for player,cost in dictOfPlayersBelongingToOtherBidders.items():
        valuesForOtherPlayersAmpl[player]=1
    for player,cost in dictOfMyPlayers.items():
        budget-=cost
        valuesForMyPlayersAmpl[player]=1
        
    if valuesForOtherPlayersAmpl:
        ampl.getParameter("alreadyBoughtByOther").setValues(valuesForOtherPlayersAmpl)
    if valuesForMyPlayersAmpl:
        ampl.getParameter("alreadyBoughtByMe").setValues(valuesForMyPlayersAmpl)
        
        
    
    ampl.getParameter("budget").set(budget)
    ampl.solve()


In [6]:
suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders)

CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 86.36256636
31133 MIP simplex iterations
464 branch-and-bound nodes
absmipgap = 0.00842667, relmipgap = 9.75732e-05


In [7]:
buy=ampl.getVariable("putPlayerInSlot")
cost=ampl.getParameter("cost")
moduleChosen=ampl.getVariable("bidderChoosesModule")
solution=buy.getValues().toDict()
costs=cost.getValues().toDict()
modules=moduleChosen.getValues().toDict()
moduleUsed="AAA"
for moduleName,taken in modules.items():
    if taken>0.5:
        print("Module: {}\n\n".format(moduleName))
        moduleUsed=moduleName

myPls=[]
for key,val in solution.items():
    if key[0]==moduleName:
        if val>0.5:
            myPls.append(key[1])
        

Module: m442




In [8]:
for player in myPls:
    message="{} {}".format(player,costs[player])
    print(message)

ACERBI 39.66714771284779
BERISHAE 1.0
CASTROVILLI 1.0
CHIESA 44.7682376284661
CRISCITO 33.1118860902676
CRISTIANORONALDO 99.1702745533213
DALESSANDRO 1.89479143291257
HATEBOER 36.355050077803504
IZZO 41.974783627056
KUCKA 1.7897496843283898
KURTIC 29.501075982687002
LAZARO 2.55969418097252
LUKIC 1.0
MANCOSU 1.0
NKOULOU 37.59585573295389
OLSEN 2.11472259401068
PAZZINI 2.55969418097252
ROSSETTINI 1.0
SALAMON 1.0
SAMIR 4.73420119932816
SANDRO 1.0
STRAKOSHA 21.508055426361103
TOLJAN 1.0
VAVRO 2.55969418097252
VERDI 1.61249173359262
ZAPATAD 72.3340022276688
ZAPPACOSTA 2.55969418097252
ZIELINSKI 41.2591867148264
